In [1]:

PYSOLR_PATH = '/home/ubuntu/code/NFGEC/'
import sys, os
if not PYSOLR_PATH in sys.path:
    sys.path.append(PYSOLR_PATH)
    
os.chdir(PYSOLR_PATH)

import argparse
from sklearn.externals import joblib
from src.model.nn_model import Model
from src.batcher import Batcher
from src.hook import acc_hook, save_predictions

# parser = argparse.ArgumentParser()
# parser.add_argument("dataset",help="dataset to train model",choices=["figer","gillick"])
# parser.add_argument("encoder",help="context encoder to use in model",choices=["averaging","lstm","attentive"])
# parser.add_argument('--feature', dest='feature', action='store_true')
# parser.add_argument('--no-feature', dest='feature', action='store_false')
# parser.set_defaults(feature=False)
# parser.add_argument('--hier', dest='hier', action='store_true')
# parser.add_argument('--no-hier', dest='hier', action='store_false')
# parser.set_defaults(hier=False)
# args = parser.parse_args()


print "Creating the model"
model = Model(type='figer',encoder='attentive',hier=True,feature=True)

print "Loading the dictionaries"
d = "Wiki" 
dicts = joblib.load("data/"+d+"/dicts_"+"figer"+".pkl")

print "Loading the datasets"
train_dataset = joblib.load("data/"+d+"/train_"+"figer"+".pkl")
dev_dataset = joblib.load("data/"+d+"/dev_"+"figer"+".pkl")
test_dataset = joblib.load("data/"+d+"/test_"+"figer"+".pkl")

print 
print "train_size:", train_dataset["data"].shape[0]
print "dev_size: ", dev_dataset["data"].shape[0]
print "test_size: ", test_dataset["data"].shape[0]

print "Creating batchers"
# batch_size : 1000, context_length : 10
train_batcher = Batcher(train_dataset["storage"],train_dataset["data"],1000,10,dicts["id2vec"])
dev_batcher = Batcher(dev_dataset["storage"],dev_dataset["data"],dev_dataset["data"].shape[0],10,dicts["id2vec"])
test_batcher = Batcher(test_dataset["storage"],test_dataset["data"],test_dataset["data"].shape[0],10,dicts["id2vec"])

step_par_epoch = 2000


Creating the model


In [ ]:

print "start trainning"
for epoch in range(5):
    train_batcher.shuffle()
    print "epoch",epoch
    for i in range(step_par_epoch):
        context_data, mention_representation_data, target_data, feature_data = train_batcher.next()
        model.train(context_data, mention_representation_data, target_data, feature_data)
        
    print "------dev--------"
    context_data, mention_representation_data, target_data, feature_data = dev_batcher.next()
    scores = model.predict(context_data, mention_representation_data,feature_data)
    acc_hook(scores, target_data)

print "Training completed.  Below are the final test scores: "
print "-----test--------"
context_data, mention_representation_data, target_data, feature_data = test_batcher.next()
scores = model.predict(context_data, mention_representation_data, feature_data)
acc_hook(scores, target_data)
fname = "figer" + "_" + "attentive" + "_" + "true" + "_" + "true" + ".txt"
save_predictions(scores, target_data, dicts["id2label"],fname)

print "Cheers!"




start trainning
epoch 0


src/model/nn_model.py:135: FutureWarning: comparison to `None` will result in an elementwise object comparison in the future.
  if self.feature == True and feature_data != None:


------dev--------


src/model/nn_model.py:154: FutureWarning: comparison to `None` will result in an elementwise object comparison in the future.
  if self.feature == True and feature_data != None:


('     strict (p,r,f1):', (0.5703, 0.5703, 0.5703))
('loose macro (p,r,f1):', (0.8760277380952389, 0.7811042857142863, 0.8258473203091451))
('loose micro (p,r,f1):', (0.8945049504950495, 0.7624050632911392, 0.8231890660592255))
epoch 1
------dev--------
('     strict (p,r,f1):', (0.6487, 0.6487, 0.6487))
('loose macro (p,r,f1):', (0.8907764285714292, 0.8334844047619062, 0.8611785954779195))
('loose micro (p,r,f1):', (0.9048568910330751, 0.8230379746835443, 0.8620102967496742))
epoch 2
------dev--------
('     strict (p,r,f1):', (0.6889, 0.6889, 0.6889))
('loose macro (p,r,f1):', (0.8937751190476196, 0.8652646428571436, 0.8792888323796855))
('loose micro (p,r,f1):', (0.9030190514078592, 0.8619831223628692, 0.8820240485288087))
epoch 3
------dev--------
('     strict (p,r,f1):', (0.7138, 0.7138, 0.7138))
('loose macro (p,r,f1):', (0.9047506746031744, 0.8734916666666681, 0.8888464258618111))
('loose micro (p,r,f1):', (0.9143287683375437, 0.8704641350210971, 0.8918574238592396))
epoch 4


In [7]:
print "-----test--------"
context_data, mention_representation_data, target_data, feature_data = test_batcher.next()
scores = model.predict(context_data, mention_representation_data, feature_data)
acc_hook(scores, target_data)
fname = "figer" + "_" + "attentive" + "_" + "true" + "_" + "true" + ".txt"
save_predictions(scores, target_data, dicts["id2label"],fname)

print "Cheers!"



-----test--------
('     strict (p,r,f1):', (0.5914742451154529, 0.5914742451154529, 0.5914742451154529))
('loose macro (p,r,f1):', (0.7762771716146492, 0.818679692125518, 0.7969147884930902))
('loose micro (p,r,f1):', (0.717741935483871, 0.8038709677419354, 0.758368837492392))
Cheers!


In [8]:
new_dataset = joblib.load("data/Wiki/new_test_figer.pkl")


In [9]:
new_dataset_batcher = Batcher(new_dataset["storage"],new_dataset["data"],new_dataset["data"].shape[0],10,dicts["id2vec"])


In [10]:
print "-----test--------"
context_data, mention_representation_data, target_data, feature_data = new_dataset_batcher.next()
scores = model.predict(context_data, mention_representation_data, feature_data)
acc_hook(scores, target_data)
fname = "new_dataset_" + "_" + "attentive" + "_" + "true" + "_" + "true" + ".txt"
save_predictions(scores, target_data, dicts["id2label"],fname)

print "Cheers!"



-----test--------
('     strict (p,r,f1):', (0.3449775902979172, 0.3449775902979172, 0.3449775902979172))
('loose macro (p,r,f1):', (0.585438976979315, 0.6773662008963881, 0.6280566197237739))
('loose micro (p,r,f1):', (0.55805270338192, 0.7034757422157857, 0.6223823823823824))
Cheers!


In [14]:
joblib.dump(scores, "scores.plk")

['scores.plk']

In [16]:
dicts["id2label"]

{0: '/location',
 1: '/person',
 2: '/organization',
 3: '/location/city',
 4: '/person/artist',
 5: '/location/country',
 6: '/person/author',
 7: '/person/actor',
 8: '/organization/company',
 9: '/event',
 10: '/government',
 11: '/organization/sports_team',
 12: '/government/government',
 13: '/person/athlete',
 14: '/title',
 15: '/location/cemetery',
 16: '/person/musician',
 17: '/location/province',
 18: '/building',
 19: '/person/politician',
 20: '/organization/educational_institution',
 21: '/language',
 22: '/people/ethnicity',
 23: '/people',
 24: '/government_agency',
 25: '/written_work',
 26: '/location/county',
 27: '/person/director',
 28: '/living_thing',
 29: '/art',
 30: '/geography',
 31: '/product',
 32: '/art/film',
 33: '/event/military_conflict',
 34: '/military',
 35: '/broadcast_program',
 36: '/music',
 37: '/location/body_of_water',
 38: '/organization/sports_league',
 39: '/geography/island',
 40: '/news_agency',
 41: '/government/political_party',
 42: '

In [17]:
"data/"+d+"/dicts_"+"figer"+".pkl"

'data/Wiki/dicts_figer.pkl'

In [21]:
def get_true_and_prediction(scores, y_data):
    true_and_prediction = []
    for score,true_label in zip(scores,y_data):
        predicted_tag = []
        true_tag = []
        for label_id,label_score in enumerate(list(true_label)):
            if label_score > 0:
                true_tag.append(label_id)
        lid,ls = max(enumerate(list(score)),key=lambda x: x[1])
        predicted_tag.append((lid, ls))
        for label_id,label_score in enumerate(list(score)):
            if label_score > 0.5:
                if label_id != lid:
                    predicted_tag.append((label_id, label_score))
        true_and_prediction.append((true_tag, predicted_tag))
    return true_and_prediction

def acc_hook(scores, y_data):
    true_and_prediction = get_true_and_prediction(scores, y_data)
    print("     strict (p,r,f1):",strict(true_and_prediction))
    print("loose macro (p,r,f1):",loose_macro(true_and_prediction))
    print("loose micro (p,r,f1):",loose_micro(true_and_prediction))

def save_predictions(scores, y_data, id2label, fname):
    true_and_prediction = get_true_and_prediction(scores, y_data)
    with open(fname,"w") as f:
        for t, p in true_and_prediction:
            f.write(" ".join([id2label[id] for id in t]) + "\t" + " ".join(["%s:%.3f"%(id2label[id], sc) for id,sc in p]) + "\n")
    f.close()


In [22]:
save_predictions(scores, target_data, dicts["id2label"],fname)


In [20]:
%debug

> <ipython-input-18-212c71a3fe2b>(28)save_predictions()
     25     true_and_prediction = get_true_and_prediction(scores, y_data)
     26     with open(fname,"w") as f:
     27         for t, p in true_and_prediction:
---> 28             f.write(" ".join([id2label[id] for id in t]) + "\t" + " ".join(["%s:%.3f"%(id2label[id], sc) for id,sc in p]) + "\n")
     29     f.close()

ipdb> true_and_prediction
[([2, 40], [0]), ([0, 5], [3, (0, 0.92550409)]), ([76], [76, (0, 0.68891513)]), ([2], [2]), ([76], [2]), ([22], [5]), ([99], [0]), ([99], [99, (93, 0.97071743)]), ([76], [9]), ([76], [9]), ([0, 5], [12, (0, 0.74422753), (5, 0.83793867), (10, 0.98342705)]), ([0, 5], [0, (5, 0.9295665), (10, 0.95051557), (12, 0.96160907)]), ([0, 5], [10, (0, 0.76886952), (5, 0.74549156), (12, 0.9585551)]), ([99], [99, (93, 0.91035837)]), ([0, 5], [0, (5, 0.95150632), (10, 0.98280776), (12, 0.98522633)]), ([2, 12], [2, (24, 0.67305464)]), ([76], [9]), ([0, 5], [0, (5, 0.94336033), (10, 0.97073054), (12, 0.97

ipdb> true_and_prediction[0]
([2, 40], [0])
ipdb> exit
